# COMP90042 Assignment #2: Cross-language Information Retreival

```Student Name: Cong Yue
Student ID: 682020```

Please do not edit this field. It must be left as is for use in marking.

## General info

**Due date**: 5pm, Wed 25th May

**Submission method**: see LMS

**Submission materials**: completed copy of this ipython notebook. *Do not upload the corpora*, you can assume these files will be extracted into the same folder as the code.  *Do not use absolute paths* such as paths to files on your machine, use *relative paths* instead. *Include output* in the ipython notebook, and ensure your code can run sucessfully from scratch (e.g., use *Kernel -> Restart and Run All* menu option to test.)

**Late submissions**: -10% per day, no late submissions after the first week

**Marks**: 25% of mark for class

**Overview**: For this project, you'll be building a cross language information retreival system (CLIR), which is capable of searching for important text documents written in a English given a query in German, and displaying the results in German. This incorporates machine translation, information retrieval using a vector space model, and IR evaluation. A key focus of this project is critical analysis and experimental evaluation, for which you will need to report on the relative merits of various options. 

**Materials**: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages as used for workshops and the previous assignments. The main part of this assignment should be done entirely in python, using built-in python packages and the packages used in the class (NLTK, Numpy, Scipy, Matplotlib, Sci-kit Learn, and Gemsim). Please do no use any other 3rd party packages. Note that some choices of the *extended part* of the assignment may require you to work with other tools, such as 3rd party translation tools or retreival engines, in which case this part of the assignment will require working in another environment, although you should report your results in the ipython notebook and attach your other files (code, shell scripts etc) with your submission.   

You are encouraged to make use of the iPython notebooks released for this class as well as other online documentation to guide your responses, but you should not copy directly from any source. Adapting code from the class notebooks is permitted.

**Data**: The other data you will need are:
  - parallel German-English corpus.  This data comes from the Europarl corpus, which is a collection of debates held in the EU parliament over many years, translated into several EU languages. This data was collated as part of the annual WMT translation competitions. The data has been tokenised, sentence aligned using the Church-Gale algorithm. 
  - CLIR evaluation corpus, comprising a large collection of English documents sourced from Wikipedia along with several German query strings and relevance judgements for each document and query pair. 
  
The data files are as follows:
  - *bitext-small.{en,de}* is the sentence-aligned parallel corpus, of a small enough size for you to use for developing word-alignment tools. You may wish to work on a smaller subset during code development.
  - *bitext-large.{en,de}* is a much larger version of the above corpus, for language modelling and translation.
  - *newstest2013.{en,de}* is a separate small parallel corpus reserved for evaluation.
  - *dev.{docs,queries,qrel}* is a set of documents in English, queries in German and query relevance judgements, for the development of the IR components and evaluation.
  
For details of the datasets, please see:
> Philipp Koehn. *Europarl: A Parallel Corpus for Statistical Machine Translation* MT Summit 2005.   

and
> Shigehiko Schamoni, Felix Hieber, Artem Sokolov, Stefan Riezler. *Learning Translational and Knowledge-based Similarities from Relevance Rankings for Cross-Language Retrieval* ACL 2014.

You can find the PDFs for both papers online with a quick web search.

**Evaluation**: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time (less than 15 minutes on a lab desktop), and you must follow all instructions provided below, including specific implementation requirements. Indexing or translation steps might taken longer than this, however you should upload with your submission preprocessed files (e.g., text files, pickled python structures) to keep the runtime modest.
You will be marked not only on the correctness of your methods, but also on your explanation and analysis. 

Please do not change any of instruction text in the notebook. Where applicable, leave the output cells in the code, particularly when you are commenting on that output. You should add your answers and code by inserting a markdown cell between every major function or other block of code explaining its purpose or anywhere a result needs to be discussed (see the class notebooks for examples). Note that even if you do something wrong, you might get partial credit if you explain it enough that we can follow your reasoning, whereas a fully correct assignment with no text commentary will not receive a passing score. You will not be marked directly on the performance of your final classifier, but each of the steps you take to build it should be reasonable and well-defended.

**Updates**: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

**Academic Misconduct**: For most people, collaboration will form a natural part of the undertaking of this project, and we encourage you to discuss it in general terms with other students. However, it is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s [Academic Misconduct policy](http://academichonesty.unimelb.edu.au/policy.html) where inappropriate levels of collusion or plagiarism are deemed to have taken place.


### Warning: File encodings and tokenisation

All the text files are encoded in *utf-8*, which requires some care in using with python strings, the nltk tools and jupyter.  Please use the following method to convert strings into ASCII by escaping the special symbols. Be careful to do the conversion after tokenisation, as the escaped tags might interfere with the NLTK tokenizer and get treated as punctuation.

In [1]:
import nltk
from nltk.tokenize import word_tokenize

def tokenize(line, tokenizer=word_tokenize):
    utf_line = line.decode('utf-8').lower()
    return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]

tokenize("Seit damals ist er auf über 10.000 Punkte gestiegen.")

['seit',
 'damals',
 'ist',
 'er',
 'auf',
 '\\xfcber',
 '10.000',
 'punkte',
 'gestiegen',
 '.']

# Part 1: CLIR engine

The project has two parts: first you must build a CLIR engine, comprising information retrieval and translation components, and evaluate its accuracy. Next you will propose an extension to a component of the system. In each step you will need to justify your modelling and implementation decisions, and evaluate the quality of the outputs of the system, and at the end you will need to reflect on the overall outcomes.

The CLIR system will involve:
 - translating queries from German into English, the language of our text collection, using word-based translation
 - once the queries and the documents are in the same language, search over the document collection using BM25
 - evaluate the quality of ranked retreival results using the query relevance judgements

Note that you could try translating the text collection into German instead, but for this project we'll stick with translating the query.

Building the CLIR engine is the majority of work in the assignment, and  constitutes 70% of the assignment mark. Note that although there are several steps, they do not necessarily need to be attempted in a linear order. That is, some components are independent of others. If you're struggling with one component and cannot complete it, then you may be able to skip over it and return to it later. Where outputs are needed in a subsequent step you may want to consider ways of side-stepping this need, e.g., by using Google translate output rather than your system output for translating the queries. You should aim to answer all components.

## Information Retreival with BM25 (20% of assignment mark)

You should implement a vector-space retreival system using the BM25 approach. You'll need to:
 - load in the data files and tokenize the input,
 - preprocess the lexicon, e.g., by stemming and stop-word removal, 
 - calculate the TF/IDF representation for all documents in the collection,
 - store an inverted index such that you can efficiently recover the documents matching a query term, and
 - implement querying in the BM25 model
 - test that it runs with some English queries (you'll have to make these up)
 
This should run in a reasonable time over *dev.docs* (up to a few minutes to index), but beyond this does not need to be highly optimised. Feel free to use python dictionaries, sets, lists, numpy/scipy matrices etc where appropriate for best runtime, but you shouldn't need to use specialised data structures such as compression schemes or the like. Feel free to use APIs in NLTK, scikit-learn and other allowed python modules as needed (see list above.) You will probably want to save and load your index to/from disk to save repeated re-indexing every time you load the file.

In [2]:
import nltk
import string
STOPWORDS = set(nltk.corpus.stopwords.words('english'))
stemmer = nltk.stem.PorterStemmer() 
PUNCT = set(string.punctuation)

In [5]:
from nltk import word_tokenize
import numpy as np
import os
import pickle
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread
from sklearn.feature_extraction.text import TfidfVectorizer
import sys

DEV_DOC = 'dev.docs'

def extract_terms(doc):
    ''' return processed terms from a doc'''
    terms = []
    for token in word_tokenize(doc):
        if token not in STOPWORDS:
            term = stemmer.stem(token.lower())
            terms.append(term) if term not in PUNCT else 0
    return terms


def construct_tfidf(filename):
    row_vocab = {}
    docs = []
    with open(filename) as f:
        for line_no, line in enumerate(f):
            line = line.strip().split('\t')
            docid = line[0]
            row_vocab[docid] = row_vocab.get(docid, line_no)
            docs.append(line[1])
    
    vectorizer = TfidfVectorizer('content', tokenizer=extract_terms)
    tfidf = vectorizer.fit_transform(docs)
    # return row-name: row-index map; col-name: col-index map; doc-term matrix of tf-idf representation
    return row_vocab, vectorizer.vocabulary_, tfidf

def load_tfidf(row_vocab_file, col_vocab_file, tfidf_file):
    ''' load stored file of:
            row_vocab (row-name: row-index map)
            col_vocab (col-name: col-index map)
            tfidf(doc-term matrix of tf-idf representation)'''
    # check if stored file exists, if no, build them
    if os.path.isfile(row_vocab_file) and \
        os.path.isfile(col_vocab_file) and \
        os.path.isfile(tfidf_file+'.mtx'):
            row_vocab = pickle.load(open(row_vocab_file))
            col_vocab = pickle.load(open(col_vocab_file))
            tfidf = csr_matrix(mmread(tfidf_file+'.mtx'))
            
    else:
        row_vocab, col_vocab, tfidf = construct_tfidf(DEV_DOC)
        pickle.dump(row_vocab, open(row_vocab_file, 'w'))
        pickle.dump(col_vocab, open(col_vocab_file, 'w'))
        mmwrite(tfidf_file, tfidf)
        
    return row_vocab, col_vocab, tfidf

row_vocab, col_vocab, tfidf = load_tfidf('row.vocab', 'col.vocab', 'tfidf')

In [6]:
''' generate the inverted index '''

def invert(row_vocab, col_vocab, tfidf):
    inverted_dict = {}
    # invert vocabulary to id-string map
    row_map = dict((v,k) for k,v in row_vocab.iteritems())
    col_map = dict((v,k) for k,v in col_vocab.iteritems())
    # iterate over column (term)
    for col_idx in xrange(tfidf.shape[1]):
        # initialize inverted index with key to be term
        term = col_map[col_idx]
        inverted_dict[term] = inverted_dict.get(term, [])
        col_vec = tfidf[:, col_idx].toarray()
        # find indices of docs that the term emerges
        doc_indices = np.nonzero(col_vec)[0]
        # iterative over docs and add to inverted list
        for idx in doc_indices:
            doc_name = row_map[idx]
            inverted_dict[term].append(doc_name)
            
    return inverted_dict

def save_to_inverted(inv, inv_file):
    pickle.dump(inv, open(inv_file, 'w'))
    
def load_from_inverted(inv_file, args=(row_vocab, col_vocab, tfidf)):
    inv = None
    if os.path.isfile(inv_file):
        inv = pickle.load(open(inv_file))
    else:
        inv = invert(args[0], args[1], args[2])
        save_to_inverted(inv, inv_file)

    return inv

inverted_dict = load_from_inverted('inv.idx')

```your supporting explanation and analysis goes here```

## Translating the queries (40% of assignment mark)

For translation, you should implement a simple word-based translation model in a noisy channel setting, where you search for the best translation string using a language model over the English with a translation model over the back-translation of the English output string, $\vec{e}$, into the German input string, $\vec{f}$. This corresponds to finding the string, $\vec{e}$ which maximises $p(\vec{e}) p(\vec{f} | \vec{e})$. This has two components:

### Language model

The first step is to estimate a language model. You should learn both a unigram language model and a trigram language model, and compare the two. Note that the unigram will be used in the *decoding* step below. You will have to think about how to smooth your estimates, and the related problem of handling unknown words. For smoothing the trigram model, you should you use backoff smoothing. There are a few choices to be made about how much probability mass to assign to the backoff, you will need to decide how to do this -- see the lecture slides and readings for some ideas. Be careful to pad the sentence with sentinel symbols to denote the start and end of each sentence such that the model can predict the words at the start of a sentence, and those when a sentence is complete.

Please use *bitext-large.en* to train your models (start with *bitext-small.en* for your development) and evaluate the perplexity of your model on *newstest2013.en*. You should justify your development decisions and parameter settings in a markdown cell, and quantify their effect on perplexity, including the differences you notice between the unigram and the trigram models.

In [201]:
from nltk import FreqDist

def unigram_from_file(text):
    unigrams = []
    with open(text) as f:
        for line in f:
            # add sentinel symbol
            line = line.strip().decode('utf-8').lower()
            terms = ['<s>'] + extract_terms(line) + ['</s>']
            unigrams.extend(terms)
            
    return unigrams


class LanModel(object):
    def __init__(self, n, lmbda):
        self._n = n
        self._lambda = lmbda
        self._fdist = None
        self._bi_fdist = None
        self._tri_fdist = None
        self._token_length = 0
        
        # smoothing param
        self._N = {}
        self._next_count_map = []
     
    def fit(self, tokens):
        self._token_length = len(tokens)
        self._fdist = self.get_distribution(tokens, 1)
        unk_counts = len(self._fdist)
        self._fdist.setdefault('UNK', unk_counts)
        # prepare for good turing smoothing
        self._N, self._next_count_map = self.good_turing_smoothing(self._fdist)

        if self._n == 2:
            self._bi_fdist = self.get_distribution(tokens, 2)
            
        if self._n == 3:
            self._bi_fdist = self.get_distribution(tokens, 2)
            self._tri_fdist = self.get_distribution(tokens, 3)
            
            
    def good_turing_smoothing(self, fdist):
        # prepare for good turing smoothing
        # N[i] is the number of grams of count i
        N = {0:1}
        # vector that stores sorted count of every token
        vec = np.asarray(sorted(fdist.values()))
        # each position stores next available count
        # e.g. next_count_map[5] = 11, denotes next available count that bigger than 5 is 11
        next_count_map = {}
        for i, count in enumerate(vec):
            N.setdefault(count, len(vec[vec==count]))
            if i == 0:
                next_count_map.setdefault(0, count)
            elif i > 0:
                next_count_map.setdefault(vec[i-1], count)
            
        return N, next_count_map    
    

    def get_distribution(self, tokens, n):
        ''' get the distribution of n-gram'''
        fdist = None
        if n == 1:
            fdist = FreqDist(tokens)
        elif n == 2:
            fdist = FreqDist(nltk.bigrams(tokens))
        elif n == 3:
            fdist = FreqDist(nltk.trigrams(tokens))

        return fdist
    
    
    def check_for_unk(self, gram, fdist):
        return 'UNK' if fdist[gram] < 1 else gram
        
    

    def unigram_prob(self, sentence):
        '''  '''
        total_prob = 1.0
        N, next_count_map = self._N, self._next_count_map

        for word in sentence:
            word = self.check_for_unk(word, self._fdist)
            # good turing smoothing
            higher_count = next_count_map[self._fdist[word]]
            prob = ( higher_count * N[higher_count] / float(N[self._fdist[word]]) ) \
                    / self._token_length
            total_prob *= prob

        return total_prob


    def bigram_prob(self, sentence, ):
        ''' backoff + interpolation'''
        sent_grams = nltk.bigrams(sentence)
        total_prob = 1.0

        for gram in sent_grams:
            # avoid divide by zero, if count of unigram is zero, 
            # the count of corresponding bigram must be zero
            denominator = float(self._fdist[gram[0]]) if self._fdist[gram[0]] != 0 else 1.0
            
            prob = self._lambda * ( self._bi_fdist[gram] / denominator ) \
                    + (1-self._lambda) * self.unigram_prob([gram[1]]) 
            total_prob *= prob

        return total_prob
    

    def trigram_prob(self, sentence, ):
        ''' backoff + interpolation'''
        sent_grams = nltk.trigrams(sentence)
        total_prob = 1.0

        for gram in sent_grams:
            # avoid divide by zero, if count of bigram is zero, 
            # then the count of corresponding trigram will always be zero
            denominator = float(self._bi_fdist[gram[:-1]]) if self._bi_fdist[gram[:-1]] != 0 else 1.0
            
            prob = self._lambda * ( self._tri_fdist[gram] / denominator ) \
                    + (1-self._lambda) * self.bigram_prob(gram[1:])
            total_prob *= prob

        return total_prob
    
    
    def get_log_proba(self, sentence, ):
        log_proba = 0.0
        if self._n == 1:
            log_proba = np.log(self.unigram_prob(sentence))
        if self._n == 2:
            log_proba = np.log(self.bigram_prob(sentence))
        if self._n == 3:
            log_proba = np.log(self.trigram_prob(sentence))
            
        return log_proba
    
    
    def get_proba(self, sentence,):
        proba = 0.0
        if self._n == 1:
            proba = self.unigram_prob(sentence)
        if self._n == 2:
            proba = self.bigram_prob(sentence)
        if self._n == 3:
            proba = self.trigram_prob(sentence)
            
        return proba


def fetch_sentence(test_file):
    ''' ignore unknown words, because unknown word will be smoothed 
        in the phase of probability calculation'''
    with open(test_file) as f:
        for line in f:
            line = line.strip().decode('utf-8').lower()
            terms = ['<s>'] + extract_terms(line) + ['</s>']
            yield terms
            
    
def get_perplexity(test_file, model):
    total_log_prob = 0.0
    test_token_count = 0
    for sentence in fetch_sentence(test_file):
        total_log_prob += model.get_log_proba(sentence)
        test_token_count += len(sentence)
    return np.exp(-total_log_prob/test_token_count)
   
    
import sys
train_file = 'bitext-small.en'
test_file = 'newstest2013.en'
train_tokens = unigram_from_file(train_file)
#uni_model = LanModel(1, None)
#uni_model.fit(train_tokens)
#tri_model = LanModel(3, 0.45)
#tri_model.fit(train_tokens)
# get_perplexity(test_file, uni_model)
#tri_model = LanModel(3, None)
#tri_model.fit(train_tokens)
for lmbda in [0.9, 0.5, 0.1, 1e-2, 1e-4, 1e-8]:
    tri_model._lambda = lmbda
    print get_perplexity(test_file, tri_model)

    

9247.1956551
875.152690721
415.373263023
404.534566277
423.870530106
425.703477488


In [163]:
doc = ['<s>', 'we', 'do', 'use', 'it', '</s>']
doc2 = ['we', 'do', 'use', 'it']
print uni_model.get_log_proba(doc)
print uni_model.get_proba(doc)
#fdist = FreqDist(doc2)
#fdist.keys()
1060.11664426
import warnings
warnings.filterwarnings('error')

-26.0228768673
4.99353586316e-12


In [188]:
doc = ['<s>', 'we', 'do', 'we', 'we', 'it', '</s>']
fdist = FreqDist(doc)
sorted(fdist.values())
2.66775686128e-319*3.44173455371e-07
np.log(2.66775686128e-325)

RuntimeWarning: divide by zero encountered in log

### Translation model

The next step is to estimate translation model probabilities. For this we will use a word-based alignment model, e.g., *ibm1* to learn word-based translation probabilities using expectation maxisation. Your task is to obtain good quality word alignments from this data, which will require careful use of the word alignment models. You will want to training in both translation directions, and combining the alignments.

You should use the *bitext-small* files for this purpose, and be aware that it may take a minute or two to train ibm1 on this data. Inspect some of the word alignments and translation probabilities (e.g., using a few common words in German such as *haus*) to see if your approach is working, and give some examples of output alignments that you find that are good and bad. 

In [ ]:
your code goes here

### Decoding 

Finally you'll need to solve for the best translation for a given string. For this you should do simple word-for-word translation where each word is translated independently. Use the alignments learned above (or the translation parameters) to translate each word of the queries into English. Compare taking the maximum probability of translation into English $p(e|f)$, with the noisy-channel probability, $p(f|e)p(e)$, which considers the reverse translation and a unigram language model probability. (You'll get a chance later to do something more ambitious, using the trigram model.)

Use your algorithms to translate the first 100 queries into English, and save them to disk.

In [ ]:
your code goes here

Now you will need to write some text about the procedure above, and how well it worked. You should answer:
  - how good are the translations? How do your methods fare at translating the content words versus functions words? You can use Google translate to provide an alternative (pretty good) translation for comparison purposes. 
  - what are good settings for various modelling parameters such as language model discounting, translation model smoothing, any decoding parameters, and how do these affect the outputs?
  - compare the language model perplexity for the unigram and trigram models over your decoder outputs, how does the difference in perplexity between the two models compare to your test above on the monolingual data? 
  - how do the learned alignments differ between the two translation directions, and does their combination appear oto improve the predictions? (You don't need a formal evaluation here, as we have no *gold standard* alignments.)

```your text goes here (markdown); aim to write a few paragraphs here.```

***Decisions***
- Using log-probability : to avoid calculating on very small numeric value
- Smoothing: Good turing

## Putting the CLIR system together (10% of assignment mark)

Now you should couple the translation and IR components from above. Take your translated queries and use these with the IR system to find the most relevant documents. You should then evaluate these predictions against the supplied relevance judgements, using the *mean average precision* (MAP) metric. As part of this, you will want to consider tuning the settings of the IR system for best performance, and comment on how successful your approach was and evaluate the IR performance is affected by the parameter settings and modelling decisions you have made.

Note that if you were stuck on the translation steps above, or your translations are otherwise unusable, you can implement this step using google translate outputs for the 100 queries.

In [ ]:
your code goes here

```your supporting analysis goes here```

Note that in a complete CLIR you would translate the retreived documents back into German. We won't bother about that for this assignment, especially as I doubt many of you can understand German.

# Part 2: Extension and discussion

## Extension (25% of assignment mark)

You now have a working end-to-end CLIR system. The next step is to revisit some of the steps above to see if you can improve the performance of the system. You are invited to be creative with your ideas about how to do this. Note that many great ideas might not produce improvements, or end up being very difficult to implement. This will not be assessed solely on the grounds of CLIR accuracy; we will be looking primarily for interesting and creative ideas.

Here are some ideas on what you might do. Warning: some are bigger than others, aim to spend 5-6 hours in total on this.

#### Structured decoding

Decode using a higher order language model, possibly with word reordering. Using your trigram language model and a word-based or phrase-based translation model, search for the best scoring translation sentence. You may want to use the tools in *nltk.translate*, including the phrase extraction and *StackDecoder*, especially if you wish to support word reordering. The decoding algorithm is much simpler (and tractable) without reordering, so you might want to implement this yourself. You will want to work with log-probabilities to avoid numerical underflow. Note  that the *StackDecoder* is not highly stable code, so you will need to understand its code in detail if you chose to use it. You will need to supply translation log probabilities and language model log probabilities as input to the decoder and consider the context of the previous two words in estimating the probability of each word. 

#### Probabilisitc querying

Consider queries including weighted terms based on multiple translation outputs, such as the word translation probabilities or using *k-best* translations from a decoder (you will need to extract not just the best translation from the decode, but the runners up, which can be done from the stack contents after beam search.)  You should consider how best to define TF and DF for query words based on their ambiguous translations. You may find inspiration in the bibliography of [Douglas Oard](https://terpconnect.umd.edu/~oard/research.html#mlia) such as [this paper](https://terpconnect.umd.edu/~oard/pdf/coling12ture.pdf).

#### Word alignment

Implement an extended word alignment model, such as the *hmm* model (see [Vogel's paper](http://www.aclweb.org/anthology/C96-2141)) or a variant of *ibm2*. The variant of *ibm2* could include a better formulation for the distortion term, $p(i|j,I,J)$, based around rewarding values of i and j that are at similar relative positions to encourage alignments near the diagonal. This contrasts with *ibm2* which just learns a massive table of counts for all combinations of i,j,I and J. This strictly needs normalisation, as its not a probability, but for simplicity you can ignore this aspect here. This idea is inspired by [fast-align](https://github.com/clab/fast_align) which has a similar, but slightly more complex, formulation.

You might also want to experiment with [fast-align](https://github.com/clab/fast_align) (an extremely fast and accurate variant of *ibm2*) or [GIZA++](http://www.statmt.org/moses/giza/GIZA++.html) (an implementation of *ibm1* - *ibm5* and the *hmm*), both widely used and highly optimised alignment tools. Neither of these tools are in python, and may be a little involved to install and compile.

#### Decoding

Experiment with a state-of-the-art translation tool like [Moses](http://www.statmt.org/moses/).  Can you get better CLIR performance using this, trained on the full parallel dataset? You might want to consider translating the document collection into German, and then doing the IR entirely in German. You might consider building a python interface to moses, building on the existing very basic funcationality. **Warning:** Moses is a complex suite of software, and can be a little complex to install and use. However there are good installation and usage guides on the moses webpage.

#### IR engine

There are several great open source IR engines, including [Lucene](https://lucene.apache.org/core/), [Terrier](http://terrier.org/), [Zettair](http://www.seg.rmit.edu.au/zettair/) and [Lemur](http://www.lemurproject.org/). You could experiment with these, and use this to compare the accuracy of different types of IR models on the CLIR data. Alternatively, you could implement a faster or more compact retreival system in python, e.g., using compressed vbyte encodings, skip lists or similar.

Note that the development of some of the extensions maye require working in the command shell and in other languages than python. If this is the case, you should attach your code in separate files as part of your submission, but include the analysis text here. *Note that excellent submissions implementing interfaces or models not currently in NLTK will be considered for inclusion into the toolkit.* You should include both the code, and text explaining your work and findings.

In [ ]:
your code goes here

```your supporting description analysis goes here```

## Discussion (5% of marks)

What conclusions can you make about CLIR, or more generally translation and IR? Overall what approaches worked and what ones didn't? What insights do you have from doing this, put another way, if you were to start again, what approach would you take and why?

```your analysis text goes here; aim to write a paragraph or two```